In [1]:
import torch

def f(x):
    s = 10.0
    return 1. / (1. + torch.exp(-s*x))

def get_pm(x, ij_list):
    n = x.shape[0]
    P = torch.eye(n, dtype=x.dtype)
    for i, j in ij_list:
        alpha_ij = f(x[j] - x[i])
        P[i, i] = alpha_ij
        P[j, j] = alpha_ij
        P[i, j] = 1 - alpha_ij
        P[j, i] = 1 - alpha_ij
    return P

def mprint(*args):
    for x in args:
        print(x)

In [2]:
x = torch.tensor([3, 2, 4, 1], dtype=torch.float32)

In [3]:
# layer1 
P1 = get_pm(x, ij_list=[(0, 2), (1,3)])
P = P1
sx = P @ x
mprint(sx, P, P.sum())

tensor([3.0000, 1.0000, 4.0000, 2.0000])
tensor([[9.9995e-01, 0.0000e+00, 4.5419e-05, 0.0000e+00],
        [0.0000e+00, 4.5398e-05, 0.0000e+00, 9.9995e-01],
        [4.5419e-05, 0.0000e+00, 9.9995e-01, 0.0000e+00],
        [0.0000e+00, 9.9995e-01, 0.0000e+00, 4.5398e-05]])
tensor(4.)


In [4]:
# layer2
P2 = get_pm(sx, ij_list=[(0, 1), (2,3)])
P = torch.matmul(P2.T, P1.T) 
sx = P @ x
mprint(sx, P, P.sum())

tensor([1.0000, 3.0000, 2.0000, 4.0000])
tensor([[2.0611e-09, 4.5398e-05, 9.3615e-14, 9.9995e-01],
        [9.9995e-01, 9.3572e-14, 4.5419e-05, 2.0611e-09],
        [9.3615e-14, 9.9995e-01, 2.0611e-09, 4.5398e-05],
        [4.5419e-05, 2.0611e-09, 9.9995e-01, 9.3572e-14]])
tensor(4.)


In [5]:
# layer3
P3 = get_pm(sx, ij_list=[(1, 2)])
P = torch.matmul(P3.T, torch.matmul(P2.T, P1.T))
sx = P @ x
mprint(sx, P, P.sum())

tensor([1.0000, 2.0000, 3.0000, 4.0000])
tensor([[2.0611e-09, 4.5398e-05, 9.3615e-14, 9.9995e-01],
        [4.5355e-05, 9.9991e-01, 4.1210e-09, 4.5396e-05],
        [9.9991e-01, 4.5355e-05, 4.5417e-05, 4.1201e-09],
        [4.5419e-05, 2.0611e-09, 9.9995e-01, 9.3572e-14]])
tensor(4.)
